<a href="https://colab.research.google.com/github/rakeshveerabhadrappa/BCG-VI/blob/main/KPMG_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np

import plotly as py
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import skew
from scipy.stats import zscore
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import datetime as dt
import matplotlib.pyplot as plt

In [30]:
urlFile = "https://raw.githubusercontent.com/rakeshveerabhadrappa/BCG-VI/main/Data/KPMG_VI_New_raw_data_update_final.xlsx"
xl_file = pd.ExcelFile(urlFile)
dfs = {}
for sheet_name in xl_file.sheet_names:
    if "Title" not in sheet_name:
        df = xl_file.parse(sheet_name)
        columnNames = df.iloc[0] 
        df = df[1:]
        df.columns = columnNames
        dfs[sheet_name] = df

In [35]:
cd_df = dfs["CustomerDemographic"]
cd_df.DOB = pd.to_datetime(cd_df["DOB"], format='%Y-%m-%d').dt.date
cd_df['AgeToDate']=pd.datetime.today().date()-cd_df.DOB
cd_df['AgeToDate_inYears'] = cd_df['AgeToDate']/ np.timedelta64(1, 'Y')
cd_df['gender'].replace(['Female', 'Femal'], 'F', inplace=True)
cd_df['gender'].replace(['Male'], 'M', inplace=True)
cd_df = cd_df.drop(columns = ['first_name', 'last_name','default'])

ca_df = dfs["CustomerAddress"]
ca_df['state'].replace('New South Wales', 'NSW', inplace=True)
ca_df['state'].replace('Victoria', 'VIC', inplace=True)
ca_df = ca_df.drop(columns = ["address", "country"])

ca_cd_df = pd.merge(cd_df, ca_df, on = "customer_id")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.



In [9]:
ca_cd_df.head()

,customer_id,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,AgeToDate,AgeToDate_inYears,postcode,state,property_valuation
0,1,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11,25213 days,69.030849,2016,NSW,10
1,2,M,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16,15286 days,41.851647,2153,NSW,10
2,4,M,33,1961-10-03,NaN,IT,Mass Customer,N,No,7,22300 days,61.055326,4211,QLD,9
3,5,F,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,8,16599 days,45.446518,2448,NSW,4
4,6,M,35,1966-09-16,NaN,Retail,High Net Worth,N,Yes,13,20491 days,56.102452,3216,VIC,9


In [36]:
tr_df = dfs["Transactions"]
tr_df['profit'] = tr_df['list_price'] - tr_df['standard_cost']

totalProfit = tr_df.groupby("customer_id")["profit"].agg(['sum','count'])
totalProfit.rename(columns={
    'sum': 'monetary', 
    'count': 'frequency', 
    }, inplace=True
)

most_recent_purchase = tr_df['transaction_date'].max()
tr_df['last_purchase_days_ago'] = most_recent_purchase - tr_df['transaction_date']
tr_df['last_purchase_days_ago'] /= np.timedelta64(1, 'D')

recentPurchaseDf = tr_df.groupby("customer_id")["last_purchase_days_ago"].agg(['min'])
recentPurchaseDf.rename(columns={'min': 'recency'}, inplace=True)

In [37]:
#ca_cd_df = ca_cd_df.dropna()
ca_cd_df = pd.merge(ca_cd_df, totalProfit, on = "customer_id")
ca_cd_df = pd.merge(ca_cd_df, recentPurchaseDf, on = "customer_id")
ca_cd_df.monetary = ca_cd_df.monetary.astype('float')
ca_cd_df.past_3_years_bike_related_purchases = ca_cd_df.past_3_years_bike_related_purchases.astype('int')
ca_cd_df.property_valuation = ca_cd_df.property_valuation.astype('int')
#ca_cd_df.tenure = ca_cd_df.tenure.astype('int')

In [29]:
ca_cd_df.describe(include='all')

,customer_id,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,AgeToDate,AgeToDate_inYears,postcode,state,property_valuation,monetary,frequency,recency
count,2525.0,2525,2525.000000,2525,2525,2525,2525,2525,2525,2525.000000,2525,2525.000000,2525.0,2525,2525.000000,2525.000000,2525.000000,2525.000000
unique,2525.0,3,NaN,2324,195,9,3,2,2,NaN,NaN,NaN,762.0,3,NaN,NaN,NaN,NaN
top,1.0,F,NaN,1978-01-30,Social Worker,Manufacturing,Mass Customer,N,Yes,NaN,NaN,NaN,2170.0,NSW,NaN,NaN,NaN,NaN
freq,1.0,1286,NaN,6,36,615,1249,2524,1288,NaN,NaN,NaN,20.0,1346,NaN,NaN,NaN,NaN
mean,NaN,NaN,49.411089,NaN,NaN,NaN,NaN,NaN,NaN,10.722772,16434 days 08:10:27.326732800,44.995696,NaN,NaN,7.480000,3130.171826,5.667327,61.280000
std,NaN,NaN,28.602648,NaN,NaN,NaN,NaN,NaN,NaN,5.682986,4664 days 23:08:12.707404032,12.772238,NaN,NaN,2.832363,1792.023295,2.298500,58.504355
min,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,7531 days 00:00:00,20.619178,NaN,NaN,1.000000,15.080000,1.000000,0.000000
25%,NaN,NaN,25.000000,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,12941 days 00:00:00,35.431255,NaN,NaN,6.000000,1835.260000,4.000000,18.000000
50%,NaN,NaN,48.000000,NaN,NaN,NaN,NaN,NaN,NaN,11.000000,16460 days 00:00:00,45.065949,NaN,NaN,8.000000,2862.410000,6.000000,44.000000
75%,NaN,NaN,73.000000,NaN,NaN,NaN,NaN,NaN,NaN,16.000000,19782 days 00:00:00,54.161276,NaN,NaN,10.000000,4164.210000,7.000000,85.000000


In [47]:
rfmRef = ca_cd_df[['monetary','frequency', 'recency']].describe()
rfmRef_forAnalysis = rfmRef.iloc[3:]
rfmRef_forAnalysis


,monetary,frequency,recency
min,15.08,1.0,0.0
25%,1841.25,4.0,17.0
50%,2859.96,6.0,44.0
75%,4181.00,7.0,85.0
max,11668.95,14.0,353.0


In [46]:
def rankRecency(x, p, d):
    print(f"x = {x}, d= {d}\n p= {p} \n d[p] = {d[p]} \n d[p][0.25]= {d[p][0.25]}")
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.5]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else: 
        return 4

def rankFM(x, p, d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.5]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else: 
        return 1